In [0]:
from pyspark.sql.functions import col, upper, count, when

In [0]:
df_customers = spark.read.table("bronze.olist_customers")

In [0]:
df_dim_customers = df_customers.select(
    col("customer_id"),
    col("customer_unique_id").alias("unique_id"),
    col("customer_zip_code_prefix").alias("zip_prefix"),
    upper(col("customer_city")).alias("city"),
    col("customer_state").alias("state")
).dropDuplicates(["customer_id"])

## Verificações

### Nulos

In [0]:
# Verifica nulos
null_check = df_dim_customers.select([
    count(when(col(c).isNull(), c)).alias(c) for c in [
        "customer_id", "unique_id", "zip_prefix", "city", "state"
    ]
])

null_check.show()

# Se houver nulos, interrompe o notebook conforme proposto no desafio 
null_counts = null_check.collect()[0].asDict()
if any(v > 0 for v in null_counts.values()):
    raise Exception(f"Falha na verificação de qualidade: campos nulos encontrados: {null_counts}")


+-----------+---------+----------+----+-----+
|customer_id|unique_id|zip_prefix|city|state|
+-----------+---------+----------+----+-----+
|          0|        0|         0|   0|    0|
+-----------+---------+----------+----+-----+



### Duplicados

In [0]:
# Verifica se há duplicidades
duplicados = df_dim_customers.groupBy("customer_id").count().filter("count > 1")

if duplicados.count() > 0:
    raise Exception("Falha na verificação de qualidade: customer_id duplicado.")

display(duplicados)

customer_id,count


### Estados - 2 caracteres

In [0]:
# Verifica se todos os estados têm exatamente 2 caracteres
estados_invalidos = df_dim_customers.filter("LENGTH(state) != 2")

if estados_invalidos.count() > 0:
    raise Exception("Falha na verificação de qualidade: valores inválidos encontrados na coluna 'state'.")

display(estados_invalidos)

customer_id,unique_id,zip_prefix,city,state


In [0]:
%sql
CREATE DATABASE IF NOT EXISTS silver;

In [0]:
df_dim_customers.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("silver.dim_customers")